[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Bora-Ulu/Inflation-Technique/blob/main/Inflation_Technique.ipynb)

[![Binder](https://binder.pangeo.io/badge_logo.svg)](https://binder.pangeo.io/v2/gh/Bora-Ulu/Inflation-Technique/main?filepath=Inflation_Technique.ipynb)

In [1]:
import numpy as np
from itertools import permutations, combinations
import time
from scipy.sparse import coo_matrix, csc_matrix, csr_matrix
from cvxopt import matrix, solvers, sparse, spmatrix
from numba import njit
import functools 

#This is where the problem SETUP is given. As we add more utility functions, some of these inputs will eventually be computed automatically.

In [2]:
#Triangle version
#A1 A2 A3 A4 B1 B2 B3 B4 C1 C2 C3 C4
#0  1  2  3  4  5  6  7  8  9  10 11
card=4
num_var=12
num_originalvar=3
X=[[0,1,8,10],[2,3,9,11]]
Y=[[0,2,4,5],[1,3,6,7]]
Z=[[4,6,8,9],[5,7,10,11]]
inflation_order=2
anc_con=(X,Y,Z)
expr_set=np.array([1,5,9,4,8,12])-1
Data=[0.12199995751046305, 0.0022969343799089472, 0.001748319476328954, 3.999015242496535e-05, 0.028907881434196828, 0.0005736087488455967, 0.0003924033706699725, 1.1247230369521505e-05, 0.0030142577390317635, 0.09234476010282468, 4.373922921480586e-05, 0.0014533921021948346, 0.0007798079722868244, 0.024091567451515063, 1.1247230369521505e-05, 0.0003849052170902915, 0.020774884184769502, 0.000396152447459813, 0.0003049249122403608, 4.998769053120669e-06, 0.10820335492385, 0.0020794879260981982, 0.0015546171755205281, 2.4993845265603346e-05, 0.0006260958239033638, 0.020273757587194154, 7.498153579681003e-06, 0.0003374169110856452, 0.0028942872817568676, 0.08976414557915113, 2.624353752888351e-05, 0.0012984302615480939, 0.002370666223442477, 4.7488306004646356e-05, 0.0999928767540993, 0.001957018084296742, 0.0006198473625869629, 8.747845842961171e-06, 0.02636975644747481, 0.0005198719815245496, 1.4996307159362007e-05, 0.000403650601039494, 0.0005498645958432735, 0.017359475229224805, 7.123245900696953e-05, 0.002346922070440154, 0.0033754188031197316, 0.10295964618712641, 0.00038740460161685187, 7.498153579681003e-06, 0.01608353942841575, 0.000306174604503641, 0.0021319750011559654, 4.248953695152569e-05, 0.09107007399427891, 0.001860791780024169, 5.998522863744803e-05, 0.0018395470115484063, 0.002570616985567304, 0.0766411271224461, 1.874538394920251e-05, 0.00048238121362614454, 0.0006410921310627258, 0.020223769896662948]
det_assumptions=[]

In [3]:
"""
#Instrumental
#first index of A is X-copy, second is U-copy. Same for B.
#X1 X2 A11 A12 A21 A22 B11 B12 B21 B22
#0  1  2   3   4   5   6   7   8   9  
card=2
num_var=10
num_originalvar=3
ChangingX=[[0,2,3,6,7],[1,4,5,8,9]]
ChangingU=[[2,4,6,8],[3,5,7,9]]
inflation_order=2
anc_con=[ChangingX,ChangingU]
expr_set=[0,2,6,1,5,9]  #we may want to make this ai_setS plural.
det_assumptions=[[[2],[4],6,8]] #a list of four element lists, of the format 1=2 implies 3=4
Data=np.zeros((2,2,2))
Data[([0,1],0,[0,1])]=1
Data=Data.ravel()
Data
""";

In [4]:
np.unique(np.hstack((expr_set,np.arange(num_var))),return_index=True,return_inverse=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
 array([ 0,  7,  8,  3,  1, 11, 12,  4,  2, 15, 16,  5], dtype=int64),
 array([ 0,  4,  8,  3,  7, 11,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
        11], dtype=int64))

In [5]:
#@njit
#def Deduplicate(ar): #Alternatives include unique_everseen and panda.unique, see https://stackoverflow.com/a/15637512 and https://stackoverflow.com/a/41577279
#    (vals,idx,inv)=np.unique(ar,return_index=True)
#    return vals[np.argsort(idx)]
# Deduplicate(np.hstack((expr_set,np.arange(num_var))))

@njit
def MoveToFront(num_var,ar):
    return np.hstack((ar,np.delete(np.arange(num_var),ar)))

@njit
def MoveToBack(num_var,ar):
    return np.hstack((np.delete(np.arange(num_var),ar),ar))

MoveToBack(num_var,np.array(expr_set))

array([ 1,  2,  5,  6,  9, 10,  0,  4,  8,  3,  7, 11], dtype=int64)

In [6]:
#These functions can be nested inside GenerateAMatrix
def GroupGeneratorsFromSwaps(num_var,anc_con):
    def swapcols(swap):  
        pregen=np.arange(0,num_var,1,np.uint8)
        pregen[swap] = pregen[np.array([swap[1],swap[0]])]
        return pregen
    #return np.apply_along_axis(swapcols,-1,np.reshape(anc_con,(len(anc_con),-1)))
    #return np.apply_along_axis(swapcols,-1,anc_con)
    return np.array(list(swapcols(np.array(swap)) for swap in anc_con))
group_gens=GroupGeneratorsFromSwaps(num_var,anc_con)

def GroupElementsFromGenerators(group_gens):
    subsets = lambda s: tuple(np.array(list(combinations(list(s), cardinality))) for cardinality in range(2,len(s)+1))
    reindex = lambda repeatedindexing : functools.reduce(lambda x, y: x[y], repeatedindexing)
    return np.vstack((np.arange(np.shape(group_gens)[-1]),group_gens,np.vstack(list(np.array(list(reindex(elem) for elem in combos)) for combos in subsets(group_gens)))))

group_elem=GroupElementsFromGenerators(group_gens)

group_elem

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [ 2,  3,  0,  1,  4,  5,  6,  7,  9,  8, 11, 10],
       [ 1,  0,  3,  2,  6,  7,  4,  5,  8,  9, 10, 11],
       [ 0,  1,  2,  3,  5,  4,  7,  6, 10, 11,  8,  9],
       [ 3,  2,  1,  0,  6,  7,  4,  5,  9,  8, 11, 10],
       [ 2,  3,  0,  1,  5,  4,  7,  6, 11, 10,  9,  8],
       [ 1,  0,  3,  2,  7,  6,  5,  4, 10, 11,  8,  9],
       [ 3,  2,  1,  0,  7,  6,  5,  4, 11, 10,  9,  8]])

In [7]:
@njit
def GenShapedColumnIntegers(range_shape):    #Could by handy to memoize this.
    return np.arange(0,np.prod(np.array(range_shape)),1,np.int32).reshape(range_shape)

In [8]:
def MarkInvalidStrategies(card,num_var,det_assumptions):
    initialshape=tuple(np.full(num_var,card,np.uint8))
    ColumnIntegers=GenShapedColumnIntegers(initialshape)
    for detrule in det_assumptions:
        initialtranspose=MoveToFront(num_var,np.hstack(tuple(detrule)))
        inversetranspose=np.argsort(initialtranspose)
        parentsdimension=card**len(detrule[1])
        intermediateshape=(parentsdimension,parentsdimension,card,card,-1);
        ColumnIntegers=ColumnIntegers.transpose(tuple(initialtranspose)).reshape(intermediateshape)
        for i in np.arange(parentsdimension):
            for j in np.arange(card-1):
                for k in np.arange(j,card):
                    ColumnIntegers[i,i,j,k]=-1
        ColumnIntegers=ColumnIntegers.reshape(initialshape).transpose(tuple(inversetranspose))
    return ColumnIntegers

In [9]:
def ValidColumnOrbits(card, num_var, anc_con,det_assumptions=[]):
    ColumnIntegers=MarkInvalidStrategies(card,num_var,det_assumptions)
    group_elements=GroupElementsFromGenerators(GroupGeneratorsFromSwaps(num_var,anc_con))
    AMatrix=np.vstack(tuple(
        np.ravel(np.transpose(ColumnIntegers,tuple(elem))) for elem in group_elements
    ))
    print(AMatrix.shape)
    minima=np.amin(AMatrix,axis=0)
    #validminima=test[0<=minima]
    #validtest= minima==np.abs(minima)
    #AMatrix=AMatrix[:,np.unique(minima)]
    #AMatrix=AMatrix[:,minima==np.abs(minima)] #Now accounting for determinism test!
    AMatrix=np.compress(minima==np.abs(AMatrix[0]), AMatrix, axis=1)
    print(AMatrix.shape)
    return AMatrix

In [10]:
#@njit #if we are only using this function once, it does not make sense to compile with numba, even though it can be compiled. 
#Look into pandas/unique alterantive implementation?
def PositionIndex(arraywithduplicates):
    arraycopy=np.zeros_like(arraywithduplicates)
    u=np.unique(arraywithduplicates)
    arraycopy[u]=np.arange(len(u))
    return arraycopy[arraywithduplicates]


@functools.lru_cache(maxsize=16)
def GenerateEncodingMonomialToRow(original_cardinality_product,inflation_order): #I should make this recursive, as called by both A and b construction.
    return PositionIndex(np.amin(
    np.vstack(tuple(
        np.ravel(np.transpose(np.reshape(np.arange(original_cardinality_product**inflation_order),tuple(np.multiply(original_cardinality_product,np.ones(inflation_order,np.uint8)))),elem
                              )) for elem in list(permutations(np.arange(inflation_order)))
)),axis=0))
    


In [11]:
def MergeMonomials(bvector,encoding):
    return np.ravel(coo_matrix((bvector, (np.zeros(len(bvector),np.uint8), encoding)),(1, np.amax(encoding)+1)).toarray())

In [12]:
def GenerateEncodingColumnToMonomial(card,num_var,expr_set):
    initialshape=tuple(np.full(num_var,card,np.uint8))
    ColumnIntegers=GenShapedColumnIntegers(initialshape)
    #transposetuple=tuple(MoveToBack(num_var,expr_set))
    #reshapetuple=(-1,card**len(expr_set))
    #print(transposetuple)
    #print(reshapetuple)
    ColumnIntegers=ColumnIntegers.transpose(tuple(MoveToBack(num_var,np.array(expr_set)))).reshape((-1,card**len(expr_set)))
    EncodingColumnToMonomial=np.zeros(card**num_var,np.uint32)
    EncodingColumnToMonomial[ColumnIntegers]=np.arange(card**len(expr_set))
    #for i in np.arange(card**len(expr_set)):   
    #    EncodingColumnToMonomial[ColumnIntegersRearranged[:,i]]=i
    return EncodingColumnToMonomial


EncodingColumnToMonomial=GenerateEncodingColumnToMonomial(card,num_var,expr_set)
np.amax(EncodingColumnToMonomial)

4095

In [13]:
def MergeMonomials(bvector,encoding):
    return np.ravel(coo_matrix((bvector, (np.zeros(len(bvector),np.uint8), encoding)),(1, np.amax(encoding)+1)).toarray())

In [14]:
def EncodeA(card, num_var, anc_con, expr_set, inflation_order):
    original_product_cardinality=card**np.rint(len(expr_set)/inflation_order).astype(np.uint)
    EncodingMonomialToRow=GenerateEncodingMonomialToRow(original_product_cardinality,inflation_order)
    EncodingColumnToMonomial=GenerateEncodingColumnToMonomial(card,num_var,np.array(expr_set))
    return EncodingMonomialToRow[EncodingColumnToMonomial][ValidColumnOrbits(card, num_var, anc_con,det_assumptions)]

In [15]:
def FindB(Data, inflation_order):
    EncodingMonomialToRow=GenerateEncodingMonomialToRow(len(Data),inflation_order)
    preb=np.array(Data)
    b=preb
    for i in range(1,inflation_order):
        b=np.kron(preb,b)
    b=MergeMonomials(b,EncodingMonomialToRow)
    return b

In [16]:
def FormCVXOPTArrayFromOnesPositions(OnesPositions):
    columncount=OnesPositions.shape[-1]
    columnspec=np.ravel(np.broadcast_to(np.arange(columncount), (len(OnesPositions), columncount)))
    return spmatrix(np.ones(OnesPositions.size), np.ravel(OnesPositions).tolist(), columnspec.tolist(),(np.amax(OnesPositions)+1, columncount))

In [17]:
def InflationLP(EncodedA,b):
    MCVXOPT=FormCVXOPTArrayFromOnesPositions(EncodedA).T
    rowcount=MCVXOPT.size[0];
    colcount=MCVXOPT.size[1];
    CVXOPTb=matrix(np.atleast_2d(b).T)
    CVXOPTh=matrix(np.zeros((rowcount,1)))
    CVXOPTA=matrix(np.ones((1,colcount)))
    return solvers.lp(CVXOPTb,-MCVXOPT,CVXOPTh,CVXOPTA,matrix(np.ones((1,1))),solver='mosek')

In [18]:
start = time.time()
EncodedA = EncodeA(card, num_var, anc_con, expr_set, inflation_order)
print('It took', time.time()-start, 'seconds.')
print(EncodedA.shape)

(8, 16777216)
(8, 2123776)
It took 1.0029997825622559 seconds.
(8, 2123776)


In [19]:
#Data=[0.12199995751046305, 0.0022969343799089472, 0.001748319476328954, 3.999015242496535e-05, 0.028907881434196828, 0.0005736087488455967, 0.0003924033706699725, 1.1247230369521505e-05, 0.0030142577390317635, 0.09234476010282468, 4.373922921480586e-05, 0.0014533921021948346, 0.0007798079722868244, 0.024091567451515063, 1.1247230369521505e-05, 0.0003849052170902915, 0.020774884184769502, 0.000396152447459813, 0.0003049249122403608, 4.998769053120669e-06, 0.10820335492385, 0.0020794879260981982, 0.0015546171755205281, 2.4993845265603346e-05, 0.0006260958239033638, 0.020273757587194154, 7.498153579681003e-06, 0.0003374169110856452, 0.0028942872817568676, 0.08976414557915113, 2.624353752888351e-05, 0.0012984302615480939, 0.002370666223442477, 4.7488306004646356e-05, 0.0999928767540993, 0.001957018084296742, 0.0006198473625869629, 8.747845842961171e-06, 0.02636975644747481, 0.0005198719815245496, 1.4996307159362007e-05, 0.000403650601039494, 0.0005498645958432735, 0.017359475229224805, 7.123245900696953e-05, 0.002346922070440154, 0.0033754188031197316, 0.10295964618712641, 0.00038740460161685187, 7.498153579681003e-06, 0.01608353942841575, 0.000306174604503641, 0.0021319750011559654, 4.248953695152569e-05, 0.09107007399427891, 0.001860791780024169, 5.998522863744803e-05, 0.0018395470115484063, 0.002570616985567304, 0.0766411271224461, 1.874538394920251e-05, 0.00048238121362614454, 0.0006410921310627258, 0.020223769896662948]
#inflation_order=2;
#ai_naturalbitorder=[0,1,8,9,16,17,6,7,14,15,22,23]
#ai_naturalbitorder=UpgradeToBits(expr_set,2)
start = time.time()
b=FindB(Data,inflation_order)
MCVXOPT=FormCVXOPTArrayFromOnesPositions(EncodedA).T #This far the code is generalized
print('It took', time.time()-start, 'seconds.')
print(MCVXOPT.size)

It took 1.981997013092041 seconds.
(2123776, 2080)


In [20]:
#Commented out for now, as not critical to demonstration.
#solverout=InflationLP(EncodedA,b)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 2123777         
  Cones                  : 0               
  Scalar variables       : 2080            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.28            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 1.44    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization 

In [21]:
#evantually we'll need to figure out how to work with the solver output nicely
#np.rint(2*np.array(solverout['x'])).astype(np.int)